In [1]:
# %pip install python-dotenv
# %pip install gTTS
# %pip install openai
# # Install pytorch for your machine
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# %pip install pydub
# To get pydub to work, may need to   sudo apt install ffmpeg
# %pip install transformers
# %pip install xformers

In [35]:
import os
from dotenv import load_dotenv
# import pyttsx3  # text to speech
from gtts import gTTS
import gtts
# from playsound import playsound
import openai
import torch
import numpy as np
import math
from pydub import AudioSegment
from pydub.silence import detect_leading_silence
import transformers
import re

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
# Not Currently in Use Yet
# load_dotenv()
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# openai.api_key = OPENAI_API_KEY
def chatgbt(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        prompt=prompt,
        max_tokens=100
    )
    # message = response.choices[0].text.strip()
    return response

In [17]:
# Load the model
# MODEL_NAME = 'tiiuae/falcon-7b-instruct'
# 'gpt2'
# 'caffsean/t5-large-finetune-keyword-to-text-generation'  Bad
# MODEL_NAME = 'teknium/Replit-v2-CodeInstruct-3B'
# generator = transformers.pipeline('text-generation',
#                                   model=MODEL_NAME,
#                                   trust_remote_code=True,
#                                   device='cpu'
#                                   )

In [7]:
# # Models from hugging face
# PROMPT = "Write me a meditation script"
# def generate_text():
#     return generator(PROMPT, num_return_sequences=3)  # max_length=1000

# # generate_text()

In [8]:
# Just reads meditation scripts from .txt file for now
FILE_NAME = "body-scan-gpt.txt"  # Might incorporate file name as cmd line argument later
DIRECTORY_NAME = "meditation_scripts"

def get_text():
    curr_path = os.getcwd()
    dir_path = os.path.join(curr_path, DIRECTORY_NAME)
    file_path = os.path.join(dir_path, FILE_NAME)
    with open(file_path, "r") as f:
        unparsed_text = f.read()

    # Remove the text without brackets
    meditation_script = re.sub(r'\[.*?\]', '', unparsed_text)
    
    # Need to clean out the brief bracketed pauses
    return meditation_script

In [24]:
def text_to_speech(meditation_script):
    # gTTS implementation
    # speech_file_name = "speech-only1.mp3"

    # List of accents available
    accents = ['com.au', 'co.uk', 'us', 'ca', 'co.in', 'ie', 'co.za']
    speech_file_names = []

    for accent in accents:
        speech_file_name = f"gtts-{accent}.mp3"

        # Add longer pause after every period
        segments = meditation_script.split('. ')
        pause = AudioSegment.silent(duration=2000)  # milliseconds
        speech_audio = AudioSegment.empty()
        for segment in segments:
            # Create the TTS object
            tts = gTTS(segment,
                    tld=accent,    # Indian voice sounded the most soothing
                    lang="en",
                    slow=True
                    )
            tts.save('segment.mp3')
            segment_audio = AudioSegment.from_mp3('segment.mp3')
            speech_audio += segment_audio + pause

        # Save the audio file
        print(f"Saving {speech_file_name} audio file")
        path = os.path.join('gtts-voices', speech_file_name)
        speech_audio.export(path, format='mp3')

        speech_file_names.append(speech_file_name)

        os.remove('segment.mp3')  # Remove the temporary file
        
    return speech_file_names

    # pyttsx3 implementation

    # engine = pyttsx3.init() # object creation
    # rate = engine.getProperty('rate')   # getting details of current speaking rate
    # engine.setProperty('rate', 125)     # setting up new voice rate
    # volume = engine.getProperty('volume')   #getting to know current volume level (min=0 and max=1)
    # engine.setProperty('volume',1.0)    # setting up volume level  between 0 and 1
    # print("Saying script")
    # # engine.say("meditation_script")

    # Listen to different voices (only 2 options??) 
    # voices = engine.getProperty("voices")
    # engine.setProperty("voice", voices[1].id)  # Female voice sounded nicer
    # for voice in voices:
        # engine.setProperty("voice", voice.id)
    #     print(f"voice id {voice.id}")
    #     engine.say("Hello")
    # engine.runAndWait()

    # # Save to audio file
    # print("Saving pyttsx3 audio file")
    # engine.save_to_file(meditation_script, 'pyttsx3.mp3')
    # engine.runAndWait()

In [25]:
DIR_NAME = 'music_only'

def overlay_music_and_speech(speech_file_name):
    # Add music to speech

    music1 = AudioSegment.from_mp3(os.path.join(DIR_NAME, "music-only1.mp3"))
    speech = AudioSegment.from_mp3(speech_file_name)
    times_to_repeat = len(speech) / len(music1)
    # Lengthen music so it is at least the length of speech audio
    longer_music = AudioSegment.empty()
    for _ in range(math.ceil(times_to_repeat)):
        longer_music += music1

    combined = speech.overlay(longer_music)
    print("Saving combined audio")
    combined.export(f"full-{speech_file_name}", format="mp3")

In [32]:
# def main():
meditation_script = get_text()

In [34]:
speech_file_names = text_to_speech(meditation_script)

In [31]:
overlay_music_and_speech(speech_file_names[4])  # Overlay music for just the indian accent audio

NameError: name 'speech_file_names' is not defined

In [29]:
# if __name__ == "__main__":
#     main()

NameError: name 'speech_file_names' is not defined